In [ ]:
# Лабораторная работа 3 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель рекуррентной сети с цепью нейросетевых моделей управляемых рекуррентных блоков 
# с логарифмической функцией активации (гиперболический арксинус) выходного сигнала на скрытом слое
#
# 24.12.24
# Данный файл содержит код построения графиков различных зависимостей

In [1]:
import numpy as np
from gru import (
    GRUModel,
    fibonacci_generator,
    one_by_n_generator,
    plus_one_minus_one_generator,
    squared_generator,
    half_generator,
    arithmetic_progression,
    create_sliding_window_batches,
    one_half_generator,    
    mape
)
import plotly.graph_objects as go

In [2]:
# seq_length = 10
# window_size = 5
# batch_size = 2
# hidden_size = 4
# output_size = 3

# sequence = list(fibonacci_generator(seq_length))
# X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)
# X_train, y_train = X[0:int(len(X) * 0.8)], y[0:int(len(X) * 0.8)][-1]
# X_test, y_test = X[int(len(X) * 0.8):], y[int(len(X) * 0.8):][-1]

# verbosity = 1000

# max_mape = 1e-3
# learn_until_meet_mape = True

# model = GRUModel(window_size, hidden_size, output_size)

# # Параметры обучения
# lr = 1e-4
# epochs = 5000

# model.train(X_train, y_train, lr, epochs,
#             True, max_mape, verbosity)
# Проверим результат после обучения
# y_pred, _, _ = model.forward(X_train)
# print("Предсказание после обучения:\n", y_pred)
# print("Истинное значение:\n", y_train)
# print(f"MAPE: {mape(y_train, y_pred) * 100:.4f}%")

# print('TEST')
# y_pred, _, _ = model.forward(X_test)
# print("Предсказания на тестовой выборке:\n", y_pred)
# print("Истинное значение на тестовой выборке:\n", y_test)
# print(f"MAPE: {mape(y_test, y_pred) * 100:.4f}%")  

In [3]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
epochs = list(range(100, 2100, 100))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_epochs: list[int] = []

seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

verbosity = 100000
lr = 1e-4

sequence = list(arithmetic_progression(seq_length, a0=1, d=1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

for max_epochs in epochs:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):              
        
        network = GRUModel(input_size=window_size, hidden_size=hidden_size,
                           output_size=output_size)
        _, _mape = network.train(x=X, y=y, lr=lr, max_epochs=max_epochs, verbosity=verbosity)
        all_epochs.append(max_epochs)
        mapes_sum += _mape        
        all_mapes.append(_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 100/100, Loss: 118.116631
MAPE: 0.769840
TRAINING FINISHED
Epoch 100/100, Loss: 122.162206
MAPE: 0.764545
TRAINING FINISHED
Epoch 100/100, Loss: 153.913630
MAPE: 0.880510
TRAINING FINISHED
Epoch 100/100, Loss: 101.699447
MAPE: 0.693915
TRAINING FINISHED
Epoch 100/100, Loss: 81.721568
MAPE: 0.625741
TRAINING FINISHED
Epoch 200/200, Loss: 38.327835
MAPE: 0.414220
TRAINING FINISHED
Epoch 200/200, Loss: 40.281731
MAPE: 0.422972
TRAINING FINISHED
Epoch 200/200, Loss: 40.705164
MAPE: 0.423409
TRAINING FINISHED
Epoch 200/200, Loss: 36.337583
MAPE: 0.390890
TRAINING FINISHED
Epoch 200/200, Loss: 37.753219
MAPE: 0.393883


KeyboardInterrupt: 

In [4]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_epochs, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=epochs, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от количества итераций обучения',
    font=dict(size=20),
    xaxis=dict(title='Количество итераций обучения'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [5]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
hidden_sizes = list(range(2, 22, 2))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 20
window_size = 5
batch_size = 2
output_size = 3

verbosity = 100000
lr = 1e-4
max_epochs = 250

sequence = list(arithmetic_progression(seq_length, a0=1, d=1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)
y = y

for size in hidden_sizes:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):              
        
        network = GRUModel(input_size=window_size, hidden_size=size,
                           output_size=output_size)
        _, _mape = network.train(x=X, y=y, lr=lr, max_epochs=max_epochs, verbosity=verbosity)
        all_sizes.append(size)
        mapes_sum += _mape        
        all_mapes.append(_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 250/250, Loss: 113.254297
MAPE: 0.728856
TRAINING FINISHED
Epoch 250/250, Loss: 114.474569
MAPE: 0.743300
TRAINING FINISHED
Epoch 250/250, Loss: 118.570898
MAPE: 0.737075
TRAINING FINISHED
Epoch 250/250, Loss: 155.171632
MAPE: 0.884375
TRAINING FINISHED
Epoch 250/250, Loss: 106.383359
MAPE: 0.709909
TRAINING FINISHED
Epoch 250/250, Loss: 75.643469
MAPE: 0.588634
TRAINING FINISHED
Epoch 250/250, Loss: 105.057270
MAPE: 0.715384
TRAINING FINISHED
Epoch 250/250, Loss: 70.417936
MAPE: 0.573706
TRAINING FINISHED
Epoch 250/250, Loss: 126.115568
MAPE: 0.783584
TRAINING FINISHED
Epoch 250/250, Loss: 72.562564
MAPE: 0.579172
TRAINING FINISHED
Epoch 250/250, Loss: 59.682925
MAPE: 0.518430
TRAINING FINISHED
Epoch 250/250, Loss: 41.986958
MAPE: 0.426344
TRAINING FINISHED
Epoch 250/250, Loss: 58.819334
MAPE: 0.507957
TRAINING FINISHED
Epoch 250/250, Loss: 61.345379
MAPE: 0.529299
TRAINING FINISHED
Epoch 250/250, Loss: 42.006700
MAPE: 0.421710
TRAINING FINISHED
Epoch 250/250, 

In [6]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=hidden_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>количества нейронов на скрытом слое',
    font=dict(size=20),
    xaxis=dict(title='Количество нейронов на скрытом слое'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [14]:
# MAPE relation with sliding window
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()


all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 50
window_sizes = list(range(1, seq_length // 2))
hidden_size = 10
batch_size = 2
output_size = 3

verbosity = 100000
max_mape = 1e-3
learn_until_meet_mape = True
lr = 1e-4
max_epochs = 1000

sequence = list(arithmetic_progression(seq_length, a0=1, d=1))

for size in window_sizes:
    X, y = create_sliding_window_batches(sequence, size, batch_size, output_size)    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):              
        
        network = GRUModel(input_size=size, hidden_size=hidden_size,
                           output_size=output_size)
        _, _mape = network.train(x=X, y=y, lr=lr, max_epochs=max_epochs, verbosity=verbosity)
        all_sizes.append(size)
        mapes_sum += _mape        
        all_mapes.append(_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 1000/1000, Loss: 29.783604
MAPE: 0.285755
TRAINING FINISHED
Epoch 1000/1000, Loss: 27.544617
MAPE: 0.278742
TRAINING FINISHED
Epoch 1000/1000, Loss: 28.025855
MAPE: 0.277098
TRAINING FINISHED
Epoch 1000/1000, Loss: 32.704714
MAPE: 0.288321
TRAINING FINISHED
Epoch 1000/1000, Loss: 32.037401
MAPE: 0.285830
TRAINING FINISHED
Epoch 1000/1000, Loss: 22.679322
MAPE: 0.252511
TRAINING FINISHED
Epoch 1000/1000, Loss: 24.479679
MAPE: 0.244326
TRAINING FINISHED
Epoch 1000/1000, Loss: 27.658556
MAPE: 0.260378
TRAINING FINISHED
Epoch 1000/1000, Loss: 19.930330
MAPE: 0.246136
TRAINING FINISHED
Epoch 1000/1000, Loss: 25.090126
MAPE: 0.242071
TRAINING FINISHED
Epoch 1000/1000, Loss: 27.196415
MAPE: 0.253209
TRAINING FINISHED
Epoch 1000/1000, Loss: 18.520941
MAPE: 0.217855
TRAINING FINISHED
Epoch 1000/1000, Loss: 19.811121
MAPE: 0.210037
TRAINING FINISHED
Epoch 1000/1000, Loss: 20.008474
MAPE: 0.223217
TRAINING FINISHED
Epoch 1000/1000, Loss: 21.651516
MAPE: 0.225296
TRAINING F

In [15]:
# Plotting results
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=window_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>размера скользящего окна',
    font=dict(size=20),
    xaxis=dict(title='Размер скользящего окна'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [9]:
# Настройки эксперимента
experiments_amount = 5
seq_length = 10
window_size = 3
batch_size = 2
hidden_size = 10
output_size = 3
max_epochs = 2000
verbosity = 100000
lr = 1e-4

# Словарь с генераторами
sequence_generators = {
    "1, 1, 2, 3, 5, 8,...": lambda n: list(fibonacci_generator(n)),
    "1, 4, 9, 16,...": lambda n: list(squared_generator(n)),
    "1, 2, 3, 4, 5,...": lambda n: list(arithmetic_progression(n, a0=1, d=1)),    
    "1, 0.5 ,0.(3), 0.25, 0.2,...": lambda n: list(one_by_n_generator(n)),
    "1, -1, 1, -1,...": lambda n: list(plus_one_minus_one_generator(n)),
    "1, 0.5, 1, 0.5, 0.5, 1,...": lambda n: list(one_half_generator(n)),
}

all_mapes = []
mapes_mean = []
all_sequences = []

# Тренировка для каждой последовательности
for seq_name, seq_generator in sequence_generators.items():
    sequence = seq_generator(seq_length)
    print(sequence)
    X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

    mapes_sum = 0
    for _ in range(experiments_amount):
        network = GRUModel(input_size=window_size, hidden_size=hidden_size, output_size=output_size)
        _, _mape = network.train(x=X, y=y, lr=lr, max_epochs=max_epochs, verbosity=verbosity)
        all_sequences.append(seq_name)
        all_mapes.append(_mape)
        mapes_sum += _mape

    mapes_mean.append(mapes_sum / experiments_amount)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
TRAINING FINISHED
Epoch 2000/2000, Loss: 4.925566
MAPE: 0.211177
TRAINING FINISHED
Epoch 2000/2000, Loss: 5.384510
MAPE: 0.219949
TRAINING FINISHED
Epoch 2000/2000, Loss: 4.393614
MAPE: 0.191947
TRAINING FINISHED
Epoch 2000/2000, Loss: 4.817933
MAPE: 0.206572
TRAINING FINISHED
Epoch 2000/2000, Loss: 5.442940
MAPE: 0.222266
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
TRAINING FINISHED
Epoch 2000/2000, Loss: 1.335683
MAPE: 0.025684
TRAINING FINISHED
Epoch 2000/2000, Loss: 1.884928
MAPE: 0.031341
TRAINING FINISHED
Epoch 2000/2000, Loss: 2.697997
MAPE: 0.037323
TRAINING FINISHED
Epoch 2000/2000, Loss: 1.249500
MAPE: 0.026079
TRAINING FINISHED
Epoch 2000/2000, Loss: 1.852162
MAPE: 0.030003
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.289037
MAPE: 0.064562
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.218898
MAPE: 0.057259
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.154349
MAPE: 0.047225
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.159181


In [14]:
# Построение графика
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sequences, y=all_mapes, name="Все наблюдения", mode="markers")
)
fig.add_trace(
    go.Scatter(x=list(sequence_generators.keys()), y=mapes_mean, name="Средние значения", mode="markers")
)
fig.update_layout(
    title="Зависимость средней абсолютной ошибки<br>от вида последовательности",
    font=dict(size=20),
    xaxis=dict(title="Вид последовательности"),
    yaxis=dict(title="Средняя абсолютная ошибка"),
    width=1280,
    height=720
)
fig.show()

In [9]:
# Arithmetic
seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(arithmetic_progression(seq_length, 1, 1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")  

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.034509
MAPE: 0.012212
TEST
Предсказания на тестовой выборке:
 [18.00383565 19.17017873 20.10163977]
Истинное значение на тестовой выборке:
 [18 19 20]
MAPE: 0.4751%


In [10]:
# Fibonacci
seq_length = 15
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(fibonacci_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")

TRAINING FINISHED
Epoch 2000/2000, Loss: 401.406563
MAPE: 0.249506
TEST
Предсказания на тестовой выборке:
 [124.3206073  201.14490545 325.4805035 ]
Истинное значение на тестовой выборке:
 [144 233 377]
MAPE: 13.6679%


In [11]:
# 1 / n
seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(one_by_n_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.001185
MAPE: 0.249061
TEST
Предсказания на тестовой выборке:
 [0.06865588 0.0529477  0.07845942]
Истинное значение на тестовой выборке:
 [0.05555556 0.05263158 0.05      ]
MAPE: 27.0334%


In [12]:
# 1, -1, 1, -1, ...
seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(plus_one_minus_one_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 2000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%") 

Epoch 2000/2000, Loss: 0.083253
MAPE: 0.198552
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.083253
MAPE: 0.198552
TEST
Предсказания на тестовой выборке:
 [-0.29187779  0.14308726 -0.24763119]
Истинное значение на тестовой выборке:
 [-1  1 -1]
MAPE: 77.2468%


In [13]:
# n ** 2
seq_length = 10
window_size = 3
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(squared_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%") 

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.303113
MAPE: 0.010584
TEST
Предсказания на тестовой выборке:
 [ 64.65079107  81.64464169 100.35348175]
Истинное значение на тестовой выборке:
 [ 64  81 100]
MAPE: 0.7221%
